In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("danbooru.csv")

In [3]:
# Initialize a list to hold the new rows
new_rows = []

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    
    if pd.notna(row['alias']):
        # Split the alias string by commas
        aliases = row['alias'].split(',')
        for alias in aliases:
            new_rows.append([alias.replace("_", " "), row['category'], row['freq'], row['tag'].replace("_", " ")])
    # Append the original row with an empty redirect
    new_rows.append([row['tag'].replace("_", " "), row['category'], row['freq'], 'null'])

# Create a new DataFrame with the new rows
df = pd.DataFrame(new_rows, columns=['tag', 'category', 'freq', 'redirect'])


In [4]:
import glob

# Step 1: Read the text files from the directory using wildcards
file_pattern = 'wildcards/*.txt'  # Adjust the path as needed
files = glob.glob(file_pattern)

# Initialize an empty list to store the new rows
new_rows = []

# Step 2: Extract the Korean and English words from each line
for file in files:
    with open(file, 'r', encoding='utf-8') as f:
        for line in f:
            lst =  line.strip().split(' : ')
            if len(lst) == 1:
                continue
            korean_word, english_word = line.strip().split(' : ')
            english_word = english_word.replace('_', ' ')
            english_word = english_word.lower()
            category = file.split("/")[-1][:-4]
            category = category.replace('-','/')
            new_rows.append((category+'/'+korean_word, english_word))

# Step 4: Create a DataFrame from the new rows
new_data = pd.DataFrame(new_rows, columns=['tag', 'redirect'])

# Step 5: Merge the new data with the existing DataFrame to get category and frequency
result_df = pd.merge(new_data, df[['tag', 'category', 'freq']], left_on='redirect', right_on='tag', how='left')
result_df.drop(columns=['tag_y'], inplace=True)
result_df.rename(columns={'tag_x': 'tag'}, inplace=True)
result_df['category'].fillna(0, inplace=True)
result_df['freq'].fillna(0, inplace=True)
result_df['category'] = result_df['category'].astype(int)
result_df['freq'] = result_df['freq'].astype(int)


# Step 6: Append the new data to the original DataFrame
updated_df = pd.concat([df, result_df], ignore_index=True)


/var/folders/sk/k42rgb9s6fv2hpmyqq5jwx0h0000gn/T/ipykernel_80451/4262189807.py:31: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  result_df['category'].fillna(0, inplace=True)
/var/folders/sk/k42rgb9s6fv2hpmyqq5jwx0h0000gn/T/ipykernel_80451/4262189807.py:32: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alway

In [5]:
updated_df

,tag,category,freq,redirect
0,1girls,0,4919233,1girl
1,sole female,0,4919233,1girl
2,1girl,0,4919233,null
3,female solo,0,4092463,solo
4,solo female,0,4092463,solo
...,...,...,...,...
124727,성인/장난감/팬티 속 바이브레이터,0,1682,vibrator under panties
124728,성인/장난감/팬티호스 속 바이브레이터,0,64,vibrator under pantyhose
124729,성인/장난감/유두 위 바이브레이터,0,1068,vibrator on nipple
124730,성인/장난감/항문 속 바이브레이터,0,277,vibrator in anus


In [6]:
updated_df.to_csv('db.csv', index=False, header=False)

In [149]:
combined_strings = df.apply(lambda row: row[0], axis=1)

# Calculate the length of each combined string
lengths = combined_strings.apply(len)

# Compute the average length
average_length = lengths.mean()


/var/folders/sk/k42rgb9s6fv2hpmyqq5jwx0h0000gn/T/ipykernel_32294/427097416.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  combined_strings = df.apply(lambda row: row[0], axis=1)


In [173]:
average_length

14.206792426548866

In [150]:
combined_strings = combined_strings

In [172]:
def is_subsequence(s1, s2):
    # Two pointers for s1 and s2
    i, j = 0, 0
    
    # Loop through s2
    while i < len(s1) and j < len(s2):
        # If characters match, move the pointer of s1
        if s1[i] == s2[j]:
            i += 1
        # Always move the pointer of s2
        j += 1
    
    # If we have moved through all characters of s1, it is a subsequence
    return i == len(s1)

def lcs_length_penalized(a, b):
    if a.startswith(b) or b.startswith(a):
        return 0
    m, n = len(a), len(b)
    INF = int(1e9)
    dp = [[[INF, INF] for _ in range(n+1)] for _ in range(m + 1)]
    dp[0][0][0] = 0

    for i in range(m+1):
        for j in range(n):
            if i != m and a[i] == b[j]:
                dp[i+1][j+1][1] = min(dp[i+1][j+1][1], dp[i][j][0]+1, dp[i][j][1])
            dp[i][j+1][0] = min(dp[i][j+1][0], dp[i][j][1], dp[i][j][0])
    return min(dp[m][n][1], dp[m][n][0])

def autocomplete(keyword, strings):
    # Calculate LCS length for each string compared to the keyword
    suggestions = filter(lambda x: is_subsequence(keyword, x), strings)
    print(len(list(suggestions)))
    suggestions = [(s, lcs_length_penalized(keyword, s)) for s in suggestions]
    
    # Sort suggestions based on LCS length in descending order
    sorted_suggestions = sorted(suggestions, key=lambda x: (x[1], x[0]), reverse=False)
    
    # Extract and return sorted strings
    return sorted_suggestions

# Example usage
keyword = "areol"
suggestions = autocomplete(keyword, combined_strings)[:30]
print("Autocomplete suggestions:")
for suggestion in suggestions:
    print(suggestion)

1727
Autocomplete suggestions:


In [140]:
keyword = ""
suggestions = autocomplete(keyword, combined_strings)[:30]
print("Autocomplete suggestions:")
for suggestion in suggestions:
    print(suggestion)

Autocomplete suggestions:
1girl
solo
highres
long_hair
commentary_request
breasts
looking_at_viewer
blush
smile
open_mouth
short_hair
blue_eyes
simple_background
shirt
absurdres
large_breasts
skirt
blonde_hair
multiple_girls
brown_hair
black_hair
long_sleeves
hair_ornament
white_background
1boy
gloves
red_eyes
bad_id
dress
thighhighs
